In [1]:
import os

from sources.saver.file_saver import FileSaver
from sources.parser.suspilne_parser import SuspilneParser
from sources.annotator.xml_annotator import XMLAnnotator
from sources.source.suspilne_source import SuspilneSource

saver = FileSaver(path="corpus")

for record in SuspilneSource.records():
    print(f"Processing: {record.metadata.title}")
    content = record.content()

    annotator = XMLAnnotator()
    document = SuspilneParser.parse(content, metadata=record.metadata,annotator=annotator)
    text = annotator.get_string(document)

    saver.save(text, name=record.metadata.title)
    print(f"Finished: {record.metadata.title}")





Processing: Як росіяни збирають інформацію про військовослужбовців з Полтавщини
Finished: Як росіяни збирають інформацію про військовослужбовців з Полтавщини
